In [215]:
%load_ext autoreload
%autoreload 2
import plotly.graph_objects as go
import utils
import plots
import pandas as pd
import numpy as np
pd.set_option('display.max_row', 1000)
pd.set_option('display.max_columns', 50)
data = {
    'plots': {},
    'stats': {}
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [216]:
confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

In [217]:
def transpose(df):
    df = df.groupby(['Country/Region']).sum()
    df = df.drop(['Lat', 'Long'], axis=1)
    return df.transpose()

In [218]:
confirmed = transpose(confirmed)
deaths = transpose(deaths)

In [219]:
fig = plots.confirmed_cases_over_threshold(confirmed)
fig.show()
data['plots']['figure_1'] = fig.to_json()

In [220]:
fig = plots.countries_deaths_over_threshold(deaths)
fig.show()
data['plots']['figure_2'] = fig.to_json()

In [221]:
fig = plots.top_countries_deaths_over_threshold_and_aligned(deaths)
fig.show()
data['plots']['figure_3'] = fig.to_json()

In [222]:
fig = plots.deaths_over_threshold_and_aligned(deaths)
fig.show()
data['plots']['figure_4'] = fig.to_json()

In [223]:
fig = plots.aligned_on_lockdown(deaths)
fig.show()
data['plots']['figure_5'] = fig.to_json()

In [224]:
fig = plots.deaths_area(deaths)
fig.show()
data['plots']['deaths_area'] = fig.to_json()

In [225]:
fig = plots.deaths_pie_chart(deaths)
fig.show()
data['plots']['deaths_pie_chart'] = fig.to_json()

In [226]:
fig = plots.daily_change(deaths, 3)
fig.show()
data['plots']['daily_change'] = fig.to_json()

In [227]:
fig = plots.daily_change(deaths.loc[pd.to_datetime(deaths.index) >= pd.to_datetime('2020-03-01')], 1)
fig.show()
data['plots']['daily_change2'] = fig.to_json()

In [244]:
fig = plots.animated_map(deaths, 'iso_alpha.csv')
fig.show()
data['plots']['change_map'] = fig.to_json()

In [245]:
import stats
data['stats'] = stats.calc_stats(deaths)

In [246]:
import json
with open('../../frontend/src/data/data.json', 'w') as f:
    json.dump(data, f)